In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import os
family_group = "5.Coliadinae"
annotated_genome_location = f"/mnt/h/My Drive/Circadian Rhythm Genes Project/9.CLOCK Exon Analysis/{family_group}/1.Blast_result"
species_list = os.listdir(annotated_genome_location)
if "desktop.ini" in species_list:
    species_list.remove("desktop.ini")
# species_list = ["Boloria_euphrosyne","Boloria_selene","Brenthis_daphne","Brenthis_hecate","Brenthis_ino","Dryadula_phaetusa","Dryas_iulia_moderata","Eueides_isabella","Fabriciana_adippe","Heliconius_charithonia","Heliconius_nattereri","Heliconius_sara","Philaethria_dido","Hestina_assimilis"]
# species_list = ["Limenitis_camilla"]
# species_list = ["Argynnis_bischoffii_washingtonia","Boloria_euphrosyne","Boloria_selene","Brenthis_daphne","Brenthis_hecate","Brenthis_ino","Dryadula_phaetusa","Dryas_iulia_moderata","Eueides_isabella","Fabriciana_adippe","Heliconius_charithonia","Heliconius_nattereri","Heliconius_sara","Philaethria_dido","Hestina_assimilis"]
# species_list = ["Bicyclus_anynana","Coenonympha_glycerion","Elymnias_hypermnestra","Erebia_aethiops","Erebia_ligea","Hipparchia_semele","Lasiommata_megera","Maniola_hyperantus","Maniola_jurtina","Melanargia_galathea","Oeneis_ivallda","Pararge_aegeria"]
# species_list = ["Erebia_ligea","Hipparchia_semele","Lasiommata_megera","Maniola_hyperantus","Maniola_jurtina","Melanargia_galathea","Oeneis_ivallda","Pararge_aegeria"]
# species_list = ["Vanessa_tameamea"]
for species in species_list:
    print(species)
    blast_location = f"/mnt/h/My Drive/Circadian Rhythm Genes Project/9.CLOCK Exon Analysis/{family_group}/1.Blast_result"
    
    list_of_files = os.listdir(f"{blast_location}/{species}")
    query_species_list = []
    coordinate_detail_dictionary = {}
    coordinate_info_dictionary = {}
    scaffold = ''
    for file in list_of_files:
        if file.endswith(".csv") and len(file.split("_")) >= 5 and not(file.endswith("_old.csv")) and not(file.endswith("_old_backup.csv")):
#             print(file)
            if "." in file[:-4]:
                query_species = f"{file.split('_')[-2].split('.')[1]}_{file.split('_')[-1].split('.')[0]}"
            else:
                query_species = f"{file.split('_')[-2]}_{file.split('_')[-1].split('.')[0]}"
            query_species_list.append(query_species)
            coordinate_detail_dictionary[query_species] = {}
            with open(f"{blast_location}/{species}/{file}", 'r') as coordinate_file_open:
                coordinate_file_lines_list  =  coordinate_file_open.readlines()
            if scaffold == '':
                scaffold = coordinate_file_lines_list[1].split(",")[1]
    
            for i in range(1,len(coordinate_file_lines_list)):
                
                coordinate_file_lines_split = coordinate_file_lines_list[i].split(",")
                start,stop = coordinate_file_lines_split[2],coordinate_file_lines_split[3]
                exon = f"Exon_{i}"
                if exon not in coordinate_info_dictionary:
                    coordinate_info_dictionary[exon] = {}
                coordinate_info_dictionary[exon][query_species] = [start,stop]
                coordinate_detail_dictionary[query_species][exon] = coordinate_file_lines_list[i]
            
            # break
            # print(query_species)
            # print(file)
#     print(coordinate_info_dictionary)
#     print(coordinate_detail_dictionary)
    # assert False
    output = coordinate_file_lines_list[0]
    for exon,species_specific_coordinates in coordinate_info_dictionary.items():
        # print(exon,species_specific_coordinates)
        coordinates_list = []
        previous_coordinates = ""
        match = 1
        for query_species, coordinates in coordinate_info_dictionary[exon].items():
            coordinates_list.append([query_species,coordinates])
            if previous_coordinates != '':
                if coordinates != previous_coordinates:
                    match = 0
                    
            previous_coordinates = coordinates
       
        
        
        if match == 0:
            print(exon,"mismatch")
            print(coordinates_list)
                
            while True:
                choice = input("Proceed?")
                if choice[0].lower() == "y" or choice[0].lower() == "n":
                    break
            
            while True:
                if choice[0].lower() == "y":
                    alignment_detail_dictionary = {}
                    for query_species_name in query_species_list:
                        list_of_folders  = os.listdir(f"{blast_location}/{species}")
                        for folders in list_of_folders:
                            if folders.endswith(query_species_name):
                                # print()
                                
                                with open(f"{blast_location}/{species}/{folders}/{exon}/for_blast/new_query.txt", 'r') as query_file_open:
                                    # alignment_detail = query_file_open.readlines()[0]
                                    alignment_detail = query_file_open.readlines()[0].split("set")[1][:-1]
                                    alignment_detail_dictionary[query_species_name] = alignment_detail
                                with open (f"{blast_location}/{species}/{folders}/{exon}/for_alignment/{exon}_translated_genomic_sequence_{alignment_detail}.fa.hat2", 'r') as alignment_score_file:
                                    distance_score = float(alignment_score_file.readlines()[-1].rstrip())
                                try:
                                    with open (f"{blast_location}/{species}/{folders}/{exon}/for_blast/new_query_spliced.txt", 'r') as new_query_file:
                                        new_query = (new_query_file.readlines()[-1].rstrip())
                                except:
                                    new_query = "Issue with query"
                                print(query_species_name,exon, alignment_detail, distance_score,f"\n{new_query}")
                    while True:            
                        check_alignment = input("Check Alignment??")
                        if check_alignment[0].lower() == "y" or check_alignment[0].lower() == "n":
                            break
                    if check_alignment[0].lower() == "y":
                        for query_species_name in query_species_list:
                            list_of_folders  = os.listdir(f"{blast_location}/{species}")
                            for folders in list_of_folders:
                                if folders.endswith(query_species_name):
                                    with open (f"{blast_location}/{species}/{folders}/{exon}/for_alignment/alignment_clustal_{exon}_translated_genomic_sequence_{alignment_detail_dictionary[query_species_name]}.fa.txt", 'r') as clustal_alignment_file:
                                        clustal_alignment = clustal_alignment_file.readlines()
                                    print("".join(clustal_alignment))
                                    
                                # assert False
                    while True:
                        try:
                            species_number = (input(f"Choose Species 1 - {len(query_species_list)}"))
                            
                            
                            if species_number[0].lower() == 'n':
                                output +=  f'{species},{scaffold},000,000,0,Y,Error_{exon},00,00,00\n'
                                break 
                            output += coordinate_detail_dictionary[query_species_list[int(species_number)-1]][exon]
                            break
                        except:
                            print("Retry")
                    break
                elif choice[0].lower() == "n":
                    output +=  f'{species},{scaffold},000,000,0,Y,Error_{exon},00,00,00\n'
                    break
                
                
        else:
            print(exon, "Match")
            print(coordinates_list)
            output += coordinate_detail_dictionary[query_species_list[0]][exon]
        
       

        # assert False
    # print(coordinate_detail_dictionary)
    print(output)
    with open(f"{blast_location}/{species}/final_coordinates.csv", "w") as out_file:
        out_file.write(output)
    
    # 'Coenonympha_glycerion,OY979626.1,7015875,7015963,0,N,5.Bicyclus_anynana_XM_024088150.2_Frame_0_rightoh_2_query_Exon_1,1,29,29\n'

Colias_croceus
Exon_1 Match
[['Colias_croceus', ['12997361', '12997509']], ['Zerene_cesonia', ['12997361', '12997509']], ['Leptidea_sinapis', ['12997361', '12997509']]]
Exon_2 Match
[['Colias_croceus', ['13001361', '13001509']], ['Zerene_cesonia', ['13001361', '13001509']], ['Leptidea_sinapis', ['13001361', '13001509']]]
Exon_3 Match
[['Colias_croceus', ['13002193', '13002374']], ['Zerene_cesonia', ['13002193', '13002374']], ['Leptidea_sinapis', ['13002193', '13002374']]]
Exon_4 Match
[['Colias_croceus', ['13003284', '13003401']], ['Zerene_cesonia', ['13003284', '13003401']], ['Leptidea_sinapis', ['13003284', '13003401']]]
Exon_5 Match
[['Colias_croceus', ['13005760', '13005855']], ['Zerene_cesonia', ['13005760', '13005855']], ['Leptidea_sinapis', ['13005760', '13005855']]]
Exon_6 Match
[['Colias_croceus', ['13006241', '13006298']], ['Zerene_cesonia', ['13006241', '13006298']], ['Leptidea_sinapis', ['13006241', '13006298']]]
Exon_7 Match
[['Colias_croceus', ['13006461', '13006699']], [

Choose Species 1 - 31
Exon_12 mismatch
[['Colias_croceus', ['160873', '161007']], ['Zerene_cesonia', ['160873', '161007']], ['Leptidea_sinapis', ['160873', '160989']]]
Proceed?y
Colias_croceus Exon_12 92_frame2 0.0 
WPWQEQLQRKHEELQQMIVRQQEELRQVKEQLLLARLGMLQPLIN
Zerene_cesonia Exon_12 92_frame2 0.0 
WPWQEQLQRKHEELQQMIVRQQEELRQVKEQLLLARLGMLQPLIN
Leptidea_sinapis Exon_12 92_frame2 0.128 
LQRKHEELQQMIVRQQEELRQVKEQLLLARLGMLQPLIN
Check Alignment??y
CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Colias_croceus_ WPWQEQLQRKHEELQQMIVRQQEELRQVKEQLLLARLGMLQPLIN
set92_frame2    WPWQEQLQRKHEELQQMIVRQQEELRQVKEQLLLARLGMLQPLIN
                *********************************************

CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Zerene_cesonia_ WPWQEQLQRKHEELQQMIVRQQEELRQVKEQLLLARLGMLQPLIN
set92_frame2    WPWQEQLQRKHEELQQMIVRQQEELRQVKEQLLLARLGMLQPLIN
                *********************************************

CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Leptidea_sinapi ----

Leptidea_sinapis Exon_12 22_frame0 0.172 
LQRKHEELQQMIVQQQEELRQVKEQLLLARLGMLQPLIN
Check Alignment??y
CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Colias_croceus_ WPWQEQLQRKHEELQQMIVRQQEELRQVKEQLLLARLGMLQPLIN
set22_frame0    WPWQEQLQRKHEELQQMIVQQQEELRQVKEQLLLARLGMLQPLIN
                *******************:*************************

CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Zerene_cesonia_ WPWQEQLQRKHEELQQMIVRQQEELRQVKEQLLLARLGMLQPLIN
set22_frame0    WPWQEQLQRKHEELQQMIVQQQEELRQVKEQLLLARLGMLQPLIN
                *******************:*************************

CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Leptidea_sinapi ------LQRKHEELQEMIMRQQEELRQVKEQLLLARLGILQPIIN
set22_frame0    WPWQEQLQRKHEELQQMIVQQQEELRQVKEQLLLARLGMLQPLIN
                      *********:**::******************:***:**

Choose Species 1 - 31
Exon_13 Match
[['Colias_croceus', ['2877400', '2877546']], ['Zerene_cesonia', ['2877400', '2877546']], ['Leptidea_sinapis', ['2877400', '2877546']]]
Species,

Proceed?y
Colias_croceus Exon_11 27_frame2 0.148 
SKEVPSRMEAREGDFHGRPMFAQHKAAEPAVVPQHGIGAQYLEPAPYVGVTVPGVLPILPPLPVIVSSDQAQIQ
Zerene_cesonia Exon_11 27_frame2 0.0 
QSKEVPSRMEAREGDFHGRPMFAQHKAAEPAVVPQHGIGAQYLEPAPYVGVTVPGVLPILPPLPVIVSSDQAQIQ
Leptidea_sinapis Exon_11 27_frame2 1.368 
SKEVPSRMEAREGDFHGRPMFAQHKAAEPAVVPQHGIGAQYLEPAPYVGVTVPGVLPILPPLPVIVSSDQAQIQ
Check Alignment??y
CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Colias_croceus_ ---------------------------------------SKEVPSRMEARETDFHNRTMF
set27_frame2    FKMKVYIYNNIYLTMRSSGNKLIYDNKLQMFTHNIISQQSKEVPSRMEAREGDFHGRPMF
                                                       ************ ***.*.**

Colias_croceus_ APHKAVEPAVVPQHGIGAQYLEPAPYVGVTVPGVLPILPPLPVIVSSDQAQIQ-------
set27_frame2    AQHKAAEPAVVPQHGIGAQYLEPAPYVGVTVPGVLPILPPLPVIVSSDQAQIQVCATTTD
                * ***.***********************************************       

Colias_croceus_ ---------
set27_frame2    HRSTLFGRE
                         

CLUSTAL format alignment by M

CLUSTAL format alignment by MAFFT L-INS-i (v7.525)


Leptidea_sinapi --------NKDVKTRMENTQAEANFGQGNRVSMFPQQPKIIESPEEPQHGMGAQFLDSAP
set81_frame0    KYVFLLFQNKDVKTRMENTQAEANFGQGNRVSMFPQQPKIIESPEEPQHGIGAQFLDSAP
                        ******************************************:*********

Leptidea_sinapi YARTVSVPAVMPAVMPTVLPAVVPMALPTLPVVVTSDQAQ--------
set81_frame0    YARTVSVPAVMPAVMPTVLPAVVPMALPTLPVVVTSDQAQVQMRLVYF
                ****************************************        

Choose Species 1 - 33
Exon_12 Match
[['Colias_croceus', ['22505184', '22505300']], ['Zerene_cesonia', ['22505184', '22505300']], ['Leptidea_sinapis', ['22505184', '22505300']]]
Exon_13 Match
[['Colias_croceus', ['22502242', '22502400']], ['Zerene_cesonia', ['22502242', '22502400']], ['Leptidea_sinapis', ['22502242', '22502400']]]
Species,Scaffold,Start,Stop,Complement,Error,Gene,Query_start,Query_stop,Query_Length
Leptidea_juvernica,CATIXQ010000001.1,22540106,22540254,1,N,Zerene_cesonia_XM_038360269.1_Frame_0_righ

Exon_1 Match
[['Colias_croceus', ['12973340', '12973488']], ['Zerene_cesonia', ['12973340', '12973488']], ['Leptidea_sinapis', ['12973340', '12973488']]]
Exon_2 Match
[['Colias_croceus', ['12977340', '12977488']], ['Zerene_cesonia', ['12977340', '12977488']], ['Leptidea_sinapis', ['12977340', '12977488']]]
Exon_3 Match
[['Colias_croceus', ['12979054', '12979235']], ['Zerene_cesonia', ['12979054', '12979235']], ['Leptidea_sinapis', ['12979054', '12979235']]]
Exon_4 Match
[['Colias_croceus', ['12979330', '12979447']], ['Zerene_cesonia', ['12979330', '12979447']], ['Leptidea_sinapis', ['12979330', '12979447']]]
Exon_5 Match
[['Colias_croceus', ['12989579', '12989674']], ['Zerene_cesonia', ['12989579', '12989674']], ['Leptidea_sinapis', ['12989579', '12989674']]]
Exon_6 Match
[['Colias_croceus', ['12992650', '12992704']], ['Zerene_cesonia', ['12992650', '12992704']], ['Leptidea_sinapis', ['12992650', '12992704']]]
Exon_7 Match
[['Colias_croceus', ['12993791', '12994029']], ['Zerene_cesonia